In [1]:
import os
import pandas as pd
import numpy as np

%load_ext autoreload
%autoreload 2

In [2]:
import data

In [12]:
from lightgbm import LGBMClassifier 

In [64]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix,classification_report
from sklearn.metrics import mean_squared_error

# 0. Initialize model inputs

In [81]:
# Only need to run this if you want to 
# recalculate model inputs after changing
# the inputs in the 'raw' or 'engineered' folders 
data.update_model_inputs('raw_group2')

initializing raw data
initializing engineered data
calculating model inputs
saving model inputs


In [83]:
(
    X_prior,
    X_train,
    X_test,
    y_prior,
    y_train
) = data.initialize_model_inputs(output=True)
 

X_prior.head()

,order_id,product_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id,aisle,department,unique_products,Unnamed: 0,num_users,num_times_ordered,organic,reorder_ratio
0,6,15873,22352,prior,4,1,12,30.0,Dryer Sheets Geranium Scent,75,17,laundry,household,98,15870,63,76,0,0.171053
1,6,40462,22352,prior,4,1,12,30.0,Cleanse,31,7,refrigerated,beverages,98,40459,179,322,0,0.422360
2,6,41897,22352,prior,4,1,12,30.0,Clean Day Lavender Scent Room Freshener Spray,101,17,air fresheners candles,household,98,41894,48,55,0,0.090909
3,10,1529,135442,prior,4,6,8,8.0,"Parsley, Italian (Flat), New England Grown",16,4,fresh herbs,produce,79,1528,5094,9118,0,0.420926
4,10,3464,135442,prior,4,6,8,8.0,"Organic Butterhead (Boston, Butter, Bibb) Lettuce",83,4,fresh vegetables,produce,79,3463,6204,12565,1,0.489853


In [69]:
data.product_fe.head()

,Unnamed: 0,product_id,num_users,num_times_ordered,organic,reorder_ratio
0,0,1,743,1928,0,0.589212
1,1,2,81,94,0,0.127660
2,2,3,74,283,0,0.717314
3,3,4,190,351,0,0.418803
4,4,5,6,16,0,0.562500


In [84]:
X_prior.head()

,order_id,product_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id,aisle,department,unique_products,Unnamed: 0,num_users,num_times_ordered,organic,reorder_ratio
0,6,15873,22352,prior,4,1,12,30.0,Dryer Sheets Geranium Scent,75,17,laundry,household,98,15870,63,76,0,0.171053
1,6,40462,22352,prior,4,1,12,30.0,Cleanse,31,7,refrigerated,beverages,98,40459,179,322,0,0.422360
2,6,41897,22352,prior,4,1,12,30.0,Clean Day Lavender Scent Room Freshener Spray,101,17,air fresheners candles,household,98,41894,48,55,0,0.090909
3,10,1529,135442,prior,4,6,8,8.0,"Parsley, Italian (Flat), New England Grown",16,4,fresh herbs,produce,79,1528,5094,9118,0,0.420926
4,10,3464,135442,prior,4,6,8,8.0,"Organic Butterhead (Boston, Butter, Bibb) Lettuce",83,4,fresh vegetables,produce,79,3463,6204,12565,1,0.489853


In [85]:
relevant_cols = ['order_id','product_id','order_number','order_dow','order_hour_of_day','days_since_prior_order',
                 'aisle_id','department_id','unique_products','num_users','num_times_ordered','organic','reorder_ratio']
    

# 1. Implement a model


In [86]:
# Try using the LightGBM model to make predictions 
# on our train and test datasets
# Use the code from the example below to get started

# Installation:(
    # use "conda install -c conda-forge lightgbm"
    # or look here:
# Background Information: https://towardsdatascience.com/lightgbm-vs-xgboost-which-algorithm-win-the-race-1ff7dd4917d
# Simple Example: https://github.com/microsoft/LightGBM/blob/master/examples/python-guide/simple_example.py



lgb = LGBMClassifier(boosting_type = 'goss',n_estimators=10)
lgb.fit(X_prior[relevant_cols],y_prior['reordered'])



LGBMClassifier(boosting_type='goss', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=10, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [87]:
pred=lgb.predict(X_train[relevant_cols])

In [46]:
print('The rmse of prediction is:', mean_squared_error(y_train['reordered'], pred) ** 0.5)

The rmse of prediction is: 0.6070543627195373


In [68]:
X_prior.head()

,order_id,product_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id,aisle,department,unique_products,num_users
0,4,10054,178520,prior,36,1,9,7.0,Kellogg's Nutri-Grain Apple Cinnamon Cereal,48,14,breakfast bars pastries,breakfast,108,544
1,4,17616,178520,prior,36,1,9,7.0,Goldfish Cheddar Baked Snack Crackers,78,19,crackers,snacks,108,3142
2,4,21351,178520,prior,36,1,9,7.0,Nutri-Grain Soft Baked Strawberry Cereal Break...,48,14,breakfast bars pastries,breakfast,108,1026
3,4,22598,178520,prior,36,1,9,7.0,Kellogg's Nutri-Grain Blueberry Cereal,48,14,breakfast bars pastries,breakfast,108,385
4,4,25146,178520,prior,36,1,9,7.0,Original Orange Juice,31,7,refrigerated,beverages,108,9507


In [50]:
feature_importance = pd.DataFrame({
    'columns':relevant_cols,
    'feature_importance': lgb.feature_importances_ / sum(lgb.feature_importances_)
}).sort_values('feature_importance',ascending=False)


In [51]:
feature_importance

,columns,feature_importance
7,department_id,0.326667
9,num_users,0.266667
2,order_number,0.230000
6,aisle_id,0.106667
8,unique_products,0.050000
5,days_since_prior_order,0.020000
0,order_id,0.000000
1,product_id,0.000000
3,order_dow,0.000000
4,order_hour_of_day,0.000000


In [52]:
X_test.columns

Index(['order_id', 'product_id', 'user_id', 'eval_set', 'order_number',
       'order_dow', 'order_hour_of_day', 'days_since_prior_order',
       'product_name', 'aisle_id', 'department_id', 'aisle', 'department',
       'unique_products', 'num_users'],
      dtype='object')

In [90]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

In [93]:
pipe = Pipeline([('classifier', RandomForestClassifier())])

# Create space of candidate learning algorithms and their hyperparameters
search_space = [{'classifier': [LogisticRegression()],
                 'classifier__penalty': ['l1', 'l2'],
                 'classifier__C': np.logspace(0, 4, 10)},
                {'classifier': [RandomForestClassifier()],
                 'classifier__n_estimators': [10, 100, 1000],
                 'classifier__max_features': [1, 2, 3]}]

In [94]:
clf_best = GridSearchCV(pipe, search_space, cv=5, verbose=0)


In [ ]:
best_model = clf_best.fit(X_prior, y_prior)

In [ ]:
best_model.best_estimator_.get_params()['classifier']

# 2. Try to calculate the following metrics for your predictions


In [ ]:

# f1, precision, accuracy, recall
# https://towardsdatascience.com/accuracy-precision-recall-or-f1-331fb37c5cb9

In [58]:
print('The rmse of prediction is:', mean_squared_error(y_train['reordered'], pred) ** 0.5)

The rmse of prediction is: 0.6070543627195373


In [89]:
print('The F1 score of prediction is:',f1_score(y_train['reordered'], pred))

The F1 score of prediction is: 0.76613980495564


In [59]:
confusion_matrix(y_train['reordered'], pred)

array([[ 16769,  94365],
       [  7829, 158350]], dtype=int64)

In [60]:
precision_score(y_train['reordered'], pred)

0.6265951763844647

In [66]:
print(classification_report(y_train['reordered'], pred))

              precision    recall  f1-score   support

           0       0.68      0.15      0.25    111134
           1       0.63      0.95      0.76    166179

    accuracy                           0.63    277313
   macro avg       0.65      0.55      0.50    277313
weighted avg       0.65      0.63      0.55    277313



In [88]:
accuracy_score(y_train['reordered'], pred)

0.6676372362178514

# Ignore: Code for zipping the code base

In [17]:
import os
import zipfile


In [18]:

def zipdir(path, ziph):
    # ziph is zipfile handle
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file))



In [19]:

zipf = zipfile.ZipFile('Python.zip', 'w', zipfile.ZIP_DEFLATED)
zipdir('data/', zipf)
zipf.close()